## Plotting Installation:

The plotting package allows you to make an interactive CRN plot. Plotting requires the [Bokeh](https://docs.bokeh.org/en/latest/docs/installation.html) and [ForceAtlas2](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0098679) libraries to be installed on your machine. Bokeh is used for plotting, ForceAtlas2 is used for graph layout. To install, type the following into the consol:

    Conda install bokeh
    pip install fa2

Alternatively, follow the installation instructions in the links above.




## Plotting Example
The CRN plot has a default representation:
* species are circles
    - orange circles are RNA species
    - blue circles are complexes
    - green circles are proteins
    - grey circles are DNA
    -there's always a purple circle that represents nothing. If your CRN doesn't have reactions that go to nothing, then it won't have any connections.
    
* reactions are squares

Click on a node (either reaction or species) and all arrows including that node will be highlighted
Mouse over a node (either reaction or species) and a tooltip will tell you what it is.

### Create a BioCRNpyler Model
Here we model a DNA assembly with a regulated promoter in a TxTl mixture.

In [5]:
%matplotlib inline

from biocrnpyler import *
import numpy as np

txtl = CRNLab("GFP")
txtl.mixture("mixture1", extract = "TxTlExtract", mixture_volume = 1e-6, mixture_parameters = 'BasicExtract.tsv')
dna = DNAassembly("mydna",promoter=RegulatedPromoter("plac",["laci"]),rbs="UTR1",protein="GFP")
txtl.add_dna(name = "G1", promoter = "pBest", rbs = "BCD2", protein = "GFP", initial_conc = 10, volume = 1e-7)
crn1 = txtl.get_model()
print(crn1.pretty_print())

Species (9) = {0. protein[RNAP], 1. protein[Ribo], 2. protein[RNAase], 3. dna[G1], 4. protein[GFP], 5. complex[protein[Ribo]:rna[G1]], 6. rna[G1], 7. complex[protein[RNAase]:rna[G1]], 8. complex[dna[G1]:protein[RNAP]]}
Reactions (6) = [
0. dna[G1] + protein[RNAP] <--> complex[dna[G1]:protein[RNAP]]        
        massaction: k_f(dna[G1],protein[RNAP])=100.0*dna[G1]*protein[RNAP]
        k_r(complex[dna[G1]:protein[RNAP]])=10.0*complex[dna[G1]:protein[RNAP]]
1. complex[dna[G1]:protein[RNAP]] --> dna[G1] + rna[G1] + protein[RNAP]        
        massaction: k_f(complex[dna[G1]:protein[RNAP]])=3.0*complex[dna[G1]:protein[RNAP]]
2. rna[G1] + protein[Ribo] <--> complex[protein[Ribo]:rna[G1]]        
        massaction: k_f(rna[G1],protein[Ribo])=10.0*rna[G1]*protein[Ribo]
        k_r(complex[protein[Ribo]:rna[G1]])=0.25*complex[protein[Ribo]:rna[G1]]
3. complex[protein[Ribo]:rna[G1]] --> rna[G1] + protein[GFP] + protein[Ribo]        
        massaction: k_f(complex[protein[Ribo]:rna[G1]])=

## Plotting a reaction graph
First we import bokeh in order to plot an interactive graph then use the graphPlot function and the BioCRNpyler.plotting.generate_networkx_graph(crn) function to produce a graph. Mouseover the circles to see which species they identify and the squares to see reactions.

In [6]:
from bokeh.models import (Plot , Range1d)
import bokeh.plotting
import bokeh.io
bokeh.io.output_notebook() #this makes the graph appear in line with the notebook
DG, DGspec, DGrxn = generate_networkx_graph(crn1) #this creates the networkx objects
plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500)) #this generates a bokeh plot
graphPlot(DG,DGspec,DGrxn,plot,layout="force",posscale=1) #now you draw the network on the plot. Layout "force" is the default. 
#"posscale" scales the entire graph. This mostly just affects the size of the arrows relative to everything else
bokeh.io.show(plot) #if you don't type this the plot won't show

#mouse over nodes to get a tooltip telling you what they are
#mouse over single lines to outline them
#click on nodes to highlight all edges that touch them
#use the magnifying glass symbol to zoom in and out
#click and drag to move around

#NOTE: this function is not deterministic in how it displays the network, because it uses randomness to push the nodes around

Loading BokehJS ...

D:\anaconda3\lib\site-packages\bokeh\models\graphs.py:164: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "


### Advanced options for generate_networkx_graph
The following options are useful changing how plotting works.

To get better control over the way reactions and species text is display using the following keywords (default values shown below):

    use_pretty_print=False
    pp_show_rates=True
    pp_show_attributes=True
    pp_show_material=True

To get better control over the colors of different nodes, use the keywords to set a species' color.
The higher keywords will take precedence.

    repr(species): "color"
    species.name: "color"
    (species.material_type, tuple(specie.attributes)): "color"
    species.material_type: "color"
    tuple(species.attributes): "color"


In [8]:
#this demonstrates the "circle" layout. reactions are in the middle with species on the outside.
#also, the pretty_print text display style

colordict = {
    "G1":"red", #will only effect the species dna_G1 and rna_G1
    "protein_GFP": "green", #will only effect the species protein_GFP
    "protein": "blue" #All protein species, protein_Ribo, protein_RNAase, and protein_RNAP will be blue
    #All other species will be grey by default. This will include all complexes.
}
DG, DGspec, DGrxn = generate_networkx_graph(crn1,
                                            colordict = colordict,
                                            use_pretty_print=True, #uses pretty print
                                            pp_show_rates=False, #this would put the reaction rates in the reaction name. It's already listed seperately in the tool tip
                                            pp_show_attributes=False, 
                                            pp_show_material = True #this lists the material of the species being displayed
)

plot = Plot(plot_width=500, plot_height=500, x_range=Range1d(-500, 500), y_range=Range1d(-500, 500)) 
graphPlot(DG,DGspec,DGrxn,plot,layout="circle",posscale=1) 
bokeh.io.show(plot) 

D:\anaconda3\lib\site-packages\bokeh\models\graphs.py:164: UserWarning: Node keys in 'layout_function' don't match node keys in the graph. These nodes may not be displayed correctly.
  warn("Node keys in 'layout_function' don't match node keys in the graph. "
